In [0]:
import pandas as pd
from pyspark.sql import *
from pyspark.sql.functions import col
import numpy as np

#Funcion de Save en Databricks
def Save_Table_Sustituir(df,DataSave,NameTable,Path):
  
  #Carpeta donde se guarda
  DELTALAKE_SILVER_PATH_Datos = Path + NameTable
  # Remove folder if it exists
  dbutils.fs.rm(DELTALAKE_SILVER_PATH_Datos, recurse=True)
  #Se guarda en formato delta
  df.write.format("delta").save(DELTALAKE_SILVER_PATH_Datos)

  D0 = str(DataSave)
  F0 = D0 + "." + NameTable
  F0 = str(F0)
  
  QUERY_0 = (f"""CREATE DATABASE IF NOT EXISTS {D0};""")
  spark.sql(QUERY_0)

  QUERY_1 = (f"""DROP TABLE IF EXISTS {F0};""")
  spark.sql(QUERY_1)

  QUERY_2 = (f"""CREATE TABLE {F0}
           USING delta
           LOCATION '{DELTALAKE_SILVER_PATH_Datos}';
           """)
  spark.sql(QUERY_2)

  print("Tabla en BD:",F0)
     
  return F0

In [0]:
# importar las 9 tablas de azure

tabla1 = 'DataSet.factura'
fac = spark.sql(f""" select * from {tabla1} """)

tabla1 = 'DataSet.infocliente'
cli = spark.sql(f""" select * from {tabla1} """)

tabla1 = 'DataSet.item'
item = spark.sql(f""" select * from {tabla1} """)

tabla1 = 'DataSet.itemcomplemento'
itemc = spark.sql(f""" select * from {tabla1} """)

tabla1 = 'DataSet.lineaordenventa'
lov = spark.sql(f""" select * from {tabla1} """)

tabla1 = 'DataSet.lineaordenventacomp'
lovc = spark.sql(f""" select * from {tabla1} """)

tabla1 = 'DataSet.mercado'
merc = spark.sql(f""" select * from {tabla1} """)

tabla1 = 'DataSet.ordenventa'
ov = spark.sql(f""" select * from {tabla1} """)

tabla1 = 'DataSet.ordenventacomp'
ovc = spark.sql(f""" select * from {tabla1} """)

In [0]:
# agregar suffix a las columnas de fac
print(" Factura  : {0:,}".format( fac.count() ))
fac = fac.select(*(col(x).alias(x + '_F') for x in fac.columns))

# agregar suffix a las columnas de lov
print(" LOV  : {0:,}".format( lov.count() ))
lov = lov.select(*(col(x).alias(x + '_LOV') for x in lov.columns))

# unión de lov a fac para crear master
master = fac.join(lov, (fac.ORNO_F == lov.ORNO_LOV) & (fac.PONO_F == lov.PONO_LOV) , how='left')
print(" master  : {0:,}".format( master.count() ))

Factura : 671,242
 LOV : 550,667
 master : 671,242

In [0]:
# agregar suffix a las columnas de ov
print(" OV  : {0:,}".format( ov.count() ))
ov = ov.select(*(col(x).alias(x + '_OV') for x in ov.columns))

# unión de orden de venta a master
master = master.join(ov, master.ORNO_F == ov.ORNO_OV, how='left')
print(" master  : {0:,}".format( master.count() ))

OV : 149,056
 master : 671,242

In [0]:
# agregar suffix a las columnas de cli
print(" Cliente  : {0:,}".format( cli.count() ))
cli = cli.select(*(col(x).alias(x + '_C') for x in cli.columns))

# unión de cliente a master
master = master.join(cli, master.CUNO_F == cli.CUNO_C, how='left')
print(" master  : {0:,}".format( master.count() ))

Cliente : 83,606
 master : 671,242

In [0]:
# agregar suffix a las columnas de merc
print(" Merc  : {0:,}".format( merc.count() ))
merc = merc.select(*(col(x).alias(x + '_M') for x in merc.columns))

# unión de mercado a master
master = master.join(merc, master.CITG_LOV == merc.CITG_M, how='left')
print(" master  : {0:,}".format( master.count() ))

Merc : 32
 master : 671,242

In [0]:
# agregar suffix a las columnas de ovc
print(" OVC  : {0:,}".format( ovc.count() ))
ovc = ovc.select(*(col(x).alias(x + '_OVC') for x in ovc.columns))

# unión de orden de venta complemento a master
master = master.join(ovc, master.ORNO_F == ovc.ORNO_OVC, how='left')
print(" master  : {0:,}".format( master.count() ))

OVC : 211,088
 master : 671,242

In [0]:
# agregar suffix a las columnas de lovc
print(" LOVC  : {0:,}".format( lovc.count() ))
lovc = lovc.select(*(col(x).alias(x + '_LOVC') for x in lovc.columns))

# unión de línea orden de venta complemento a master
master = master.join(lovc, (master.ORNO_F == lovc.ORNO_LOVC) & (master.PONO_F == lovc.PONO_LOVC), how='left')
print(" master  : {0:,}".format( master.count() ))

LOVC : 720,013
 master : 671,242

In [0]:
# agregar suffix a las columnas de item
print(" ITEM  : {0:,}".format( item.count() ))
item = item.select(*(col(x).alias(x + '_I') for x in item.columns))

# unión de ITEM a master
master = master.join(item, master.ITEM_LOV == item.ITEM_I, how='left')
print(" master  : {0:,}".format( master.count() ))

ITEM : 9,280,202
 master : 671,242

In [0]:
# agregar suffix a las columnas de itemc
print(" ITEMC  : {0:,}".format( itemc.count() ))
itemc = itemc.select(*(col(x).alias(x + '_IC') for x in itemc.columns))

# unión de ITEM a master
master = master.join(itemc, master.ITEM_LOV == itemc.ITEM_IC, how='left')
print(" master  : {0:,}".format( master.count() ))

ITEMC : 9,280,128
 master : 671,242

In [0]:
# ver cantidad de columnas de la tabla maestra
len(master.columns)

Out[11]: 813

In [0]:
# guardar la tabla maestra en databricks
Save_Table_Sustituir(master,"DataSet","masterV3","/mnt/delta/drvdev/")

Tabla en BD: DataSet.masterV3
Out[12]: 'DataSet.masterV3'

In [0]:
# agarrar la tabla ya guardada dentro de databricks
master = spark.sql(f""" select * from DataSet.masterv3""")

In [0]:
# quitar columnas que no queremos en la tabla maestra
lista = ["AMPM_OVC","AMSC_OVC","APSH_LOVC","AUPM_LOVC","BLCM_I","CDRD_LOVC","CMDS_LOVC","CONO_F","CONO_LOV","COOP_F","CPRP_I","CRAT_C","CUE1_LOVC","CUE2_LOVC","DAM2_F","DAM3_F","DAPE_OVC","DISC$2_F","DISC$2_LOV","DISC$3_F","DISC$3_LOV","EDED_C","ENWK_LOVC","FFSI_I","FONO_F","FRAC_LOVC","FRCU_LOVC","FRLA_LOVC","FROI_LOVC","GACA_OVC","HALL_I","INFQ_C","ISCN_C","LAPM_OVC","LASC_OVC","LDAM$2_LOV","LDAM$3_LOV","LENG_LOV","LIAC_LOVC","LPPM_OVC","MIMD_IC","MRPC_I","MRPO_I","NNTS_I","NOBD_I","NORA_LOVC","NPSL_I","OBA2_OVC","OBA3_OVC","OBA4_OVC","ODAM_F","ODIS_C","ODIS_OV","OMI3_OVC","OMI4_OVC","OPE3_OVC","OPE4_OVC","OPRC_I","ORIP_I","OSTA_OVC","OTC4_OVC","OTD4_OVC","OTY2_OVC","OTY4_OVC","PCPM_OVC","PCRP_I","PHBA_OVC","PHDT_OVC","PHMI_OVC","PHPE_OVC","PORC_IC","PORN_LOV","POSI_F","POSI_LOV","POTC_I","PPON_LOV","PRBK_IC","PRIC_IC","PRNO_OV","QBSI_I","QPNT_I","QUOT_I","REFCNTD_C","REFCNTD_F","REFCNTD_IC","REFCNTD_LOV","REFCNTD_LOVC","REFCNTD_M","REFCNTD_OV","REFCNTD_OVC","REFCNTU_C","REFCNTU_F","REFCNTU_IC","REFCNTU_LOV","REFCNTU_LOVC","REFCNTU_M","REFCNTU_OV","REFCNTU_OVC","REVI_LOV","STPM_I","THIC_LOV","TORN_F","TOVR$1_C","TOVR$2_C","TOVR$3_C","TOVR$4_C","TOVR$5_C","TPON_F","TRTM_C","TSRN_F","TXAM$L_F","TXTA_C","VAR2_LOVC","WIDT_LOV","SUEX_LOVC","FIDE_OVC","CCOR_C","CPPP_I","CVPS_LOV","CVQS_LOV","DMTH$1_LOV","DMTH$2_LOV","DMTH$3_LOV","ITMT_I","LSEL_LOV","RATF_F","RATF_OV","SDSC_LOV","STVA_I","ACKX_C","APPR_OVC","ASSI_I","CDEL_C","CONT_I","COPT_I","CVYN_C","DAPR_OVC","EITM_I","INRL_C","OPOL_I","PROI_I","PRRE_I","QUAL_I","REPI_I","SCOM_C","SCOM_OV","SNDR_C","SNO1_LOVC","SNP1_LOVC","SNS1_LOVC","SNV1_LOVC","WEF1_OVC","WEF2_OVC","COMP_C","CFRW_C","CFRW_OV","RATS_F","SNN1_LOVC","WPQ1_OVC","DISC$1_F","WPQ2_OVC","BQUA_F","CPCP_F","PONO_F","PONO_LOVC","WGHT_I","DQUA_F","REFCNTD_I","REFCNTU_I","CREP_C","QONO_LOVC","ORNO_F","ORNO_LOVC","ORNO_OV","ORNO_OVC","PRIC_LOVC","TXTB_OV","TXTB_OVC","OQUA_F","TXTA_I","TXTA_OVC","MATC_I","TXTA_OV","TXTA_LOV","PRIC_F","COPR_F","PADT_OVC","DDAT_F","ODAT_OV","ODAT_OVC","PRDT_OV","DEIB_OVC","CODE_OVC","CUNO_LOVC","MRKB_OVC","CLAN_C","CDEC_C","CDEC_OVC","SPTS$8_LOVC","CUNO_C","CUNO_F","CUNO_OV","CUPS_I","CUQS_I","TCQ1_OVC","TCQ2_OVC","EONO_OV","DEIA_OVC","SPTS$1_LOVC","SPTS$2_LOVC","SPTS$3_LOVC","SPTS$4_LOVC","SPTS$6_LOVC","SPTS$7_LOVC","SPTS$9_LOVC","CPAY_C","SPTS$5_LOVC","CDEL_OVC","TCQ3_OVC","CBRN_C","CCUR_C","CCUR_I","CCTY_C","CCTY_OV","CPLS_C","CWAR_I","CVAT_I","REFA_C","CCOR_OVC","CWAR_F","SNT1_LOVC","ITEM_F","ITEM_I","ITEM_IC","CITG_M","COTP_OV","COTP_OVC","MRKA_OVC","CITG_I","CREG_C","CSGS_I","ADRS_OVC","AUTO_LOVC","AUTO_OVC","CADR_C","CBAN_C","CCCO_I","CCIC_C","CCRS_C","CCRS_OV","CCUS_C","CDEL_LOV","CDIS$1_LOV","CDIS$2_LOV","CDIS$3_LOV","CLOT_LOV","CMGP_I","CNTR_F","CNTR_I","CNTR_LOV","COTP_C","CPER_OV","CPRJ_F","CPRJ_LOV","CPVA_LOV","CRTE_C","CSPD_I","CSPS_I","CTRJ_LOV","CTRJ_OV","DCNT_I","GEOC$L_C","ITEM_M","OTED_LOVC","PCGP_I","PCPR_C","PCPR_OV","PCTX_C","PCTX_OV","PLA5_OVC","PLA6_OVC","PLA7_OVC","PLDD_C","PLDD_OV","RBGP_I","SCDL_I","SFZP_OVC","RSDT_LOVC", "OSDT_LOVC", "ODDT_LOVC", "DRDT_LOVC", "DPDT_LOVC", "DTDT_LOVC", "DDCD_LOVC", "DARD_LOVC", "DRED_LOVC", "FEAD_LOVC", "DTLT_LOVC", "DNDT_LOVC", "GEDE_LOVC", "PADT_LOVC", "ICDT_LOVC", "PMSK_F", "EDCI_C", "LVDT_C", "CRED_OVC", "FEAF_IC", "FEUD_IC", "BOMD_IC", "TRDT_IC", "UFRA_I", "CZED_I", "ACTF_I", "RUNI_I", "UNOM_I", "PLMM_I", "UMER_I", "KPSL_I", "OBPR_I", "KLTC_I", "USET_I", "IDSF_OVC", "CORG_OV", "COMP_OV", "PRIO_C", "BACK_C", "ITPR_C", "OPRI_F", "DLEV$2_LOV", "DLEV$3_LOV", "TTYP_F", "BUOR_C", "BUIN_C", "ACKN_OVC", "SFCD_OVC", "CVAT_LOV", "TTYR_LOV", "CITT_LOV", "EONO_LOV", "ESEQ_LOV", "PALT_LOVC", "COTP_LOVC", "NCAJ_LOVC", "CFRW_LOVC", "BUNU_LOVC", "COTX_LOVC", "COTM_LOVC", "ITMP_LOVC", "SHST_LOVC", "CTIT_C", "PCTF_C", "REFP_C", "OSNO_C", "TELP_C", "TELX_C", "TEFX_C", "SEAK_C", "RFCI_C", "MREM_C", "CINM_C", "MSTM_C", "CRBU_C", "CPLS_OV", "REFB_OV", "CBUY_OVC", "CBYT_OVC", "CICO_OVC", "CICT_OVC", "SEAN_OVC", "SEAT_OVC", "RLSE_OVC", "CBST_OVC", "CREF_OVC", "ROUT_OVC", "DEIC_OVC", "DEID_OVC", "CREN_OVC", "CREI_OVC", "CRET_OVC", "CODI_OVC", "REFR_OVC", "REF2_OVC", "OTH1_OVC", "OTH2_OVC", "DRTC_OVC", "PPD1_OVC", "PPD2_OVC", "PPD3_OVC", "PPD4_OVC", "PPD5_OVC", "PPD6_OVC", "PPD7_OVC", "CCDE$_OVC", "CCDE$2_OVC", "CCDE$3_OVC", "CCDE$4_OVC", "CCDE$5_OVC", "CCDE$6_OVC", "CCDE$7_OVC", "CCDE$8_OVC", "OTD1_OVC", "OTD2_OVC", "OTD3_OVC", "SSEG_OVC", "PLA1_OVC", "PLA2_OVC", "PLA3_OVC", "PLA4_OVC", "ABCC_I", "CFMD_I", "SUNO_I", "CCIT_I", "UPAI_IC", "PPAI_IC", "DSPE_IC", "HVNU_IC", "LVNU_IC", "OPER_IC", "BOQI_IC", "LOGN_IC", "EPOS_LOV", "MRKA_LOVC", "MRKB_LOVC", "CTPR_LOVC", "NAME_C", "NAMD_C", "NAMC_C", "CCDE$1_OVC", "DRCT_LOV", "CSGS_LOV", "DDTA_LOV", "CUPS_LOV", "DISC$1_LOV", "DLEV$1_LOV", "CWAR_LOV", "ISSS_LOV", "SCOM_LOV", "PRDT_LOV", "PDAT_LOVC", "CSDT_LOVC", "SHCP_LOVC", "BKDT_LOVC", "OPDT_LOVC", "DIBU_LOVC", "SSLS_F", "LDAM_F", "CRLR_C", "CNPA_C", "NCIN_C", "LIDT_C", "PAYM_C", "CPAY_OV", "FCRT_OV", "CDEL_OV", "CREP_OV", "CFCG_OV", "CVYN_OV", "CDEC_OV", "DDAT_OV", "FUTO_OV", "BKYN_OV", "AGEN_OV", "SCHP_OVC", "CBTO_OVC", "FEOC_OVC", "KITM_I", "CTYP_I", "STGU_I", "ALLO_I", "LTDT_I", "OSYS_I", "OMTH_I", "FODT_I", "TMFC_I", "AUSO_I", "CPHA_I", "OQDR_I", "BFCP_I", "BFEP_I", "BFHR", "NDRP_I", "CRMP_I", "ROUN_I", "NCST_I", "CUQP_I", "LTSP_I", "LCOD_I", "MAOQ_I", "LTPP_I", "LTCP_I", "OPOL_LOV", "PLEV_LOV", "SDQU_LOVC", "MFTY_LOVC", "BKSH_LOVC", "CRYR_C", "SHCP_OVC", "PICS_I", "MAXS_I", "BFHR_I", "CCFU_I", "CMCO_LOVC", "USAF_OVC", "OQMF_I", "ECOQ_I", "OINT_I", "DDFQ_I", "CSGP_I", "DSCC_I", "SEAK_I", "SEAB_I", "USCU_I", "OLTM_I", "CUNI_I", "CPCP_I", "CPLB_I", "FOND_IC", "FREN_IC", "ALTU_IC", "FRAC_IC", "AFRA_IC", "APAR_IC", "EQCO_IC", "CPGP_I", "CPGS_I", "ORDR_I", "SRLI_LOVC", "PITM_LOVC", "FOVN_C", "CRRA_C", "TESTR_LOVC", "REQN_OVC", "BUPA_OVC", "ORBP_OVC", "COLL_OVC", "OFTA_OVC", "ODAT_LOV", "COLI_LOVC", "SPRD_LOVC", "REFC_LOVC", "DINO_F", "AMTA_LOV", "REFA_OV", "CSEL_I", "CCDE_I", "TXTS_I"]

for n in lista:
  master=master.drop(n)

In [0]:
# convertir la tabla en una tabla de pandas
masterPD = master.toPandas()

In [0]:
# replace empty cells with NAs and remove columns with NAs
import numpy as np
masterPD.replace("", np.nan, inplace=True)
masterPD.replace([0, "0"], np.nan, inplace=True)
masterPD.dropna(how="all", axis=1, inplace=True, thresh=322000)

In [0]:
# sustituir los valores dentro de la columna de US por EUA
masterPD['CCTY_LOV'] = masterPD['CCTY_LOV'].replace('US', 'EUA')
# cambiar MX por MEX y US por EUA
masterPD['CTYO_I'] = masterPD['CTYO_I'].replace('US', 'EUA')
masterPD['CTYO_I'] = masterPD['CTYO_I'].replace('MX', 'MEX')
# cambiar fecha string por type date y cambié los 0000-00-00 al año 1950
masterPD['INVD_F'] = masterPD['INVD_F'].replace('0000-00-00', '1950-01-01')
masterPD['INVD_F'] = pd.to_datetime(masterPD['INVD_F'], format= '%Y-%d-%m')
# limpieza de nombres de la variable NAMF_C
masterPD['NAMF_C'] = masterPD['NAMF_C'].replace(['MONETRREY', 'NUEVO LEON', 'NL', 'SAN NICOLAS DE LOS GARZA, N.L.', 'SAN PEDRO GARZA GARCIA,NL', 'N.L. 66450', 'N.L.', 'MONTERREY N,L.', 'GUADALUPE,NL', 'SANTA CATARINA NUEVO LEON', 'G. N.L.', 'MONTERREY,NL', 'SAN PEDRO GARZA GARCIA N.L.', 'MONTERREY, N.L. 64830', 'GARZA N.L.', 'ALLENDE,NL', 'NL.', 'APODACA,NL', 'APODACA NL,', 'SANTA CATARINA,NL', 'MOTERREY,NL', 'NL MEXICO', 'MONTERREY, N.L.', 'SAN PEDRO GARZA GARCIA', 'DE LOS GARZA NL MEXICO', 'MONTERREY NUEVO LEON', 'CAMPO MILITAR 7-A-ESCOBEDO NL', 'SAN NICOLAS DE LOS GARZA', 'MEXICO, CP 65550', 'COL. PRIMAVERA C.P 64830', '64180', '66484', 'C.P 66230', '66450', '66600', '66057', '66260', "64000", "66635", "66650", "C.P 66260", "C.P 64700", '66270', '66615', '66266', '64440', '64463', '66350', 'C.P 64000', '64710', '64460', '64780', '64410', 'CP.64000', 'C.P 66414', '66220', 'C.P 67180', '80129'], 'NLE')
# el DF incluye todo lo del estado de México. se asume que las delegaciones también son del DF
masterPD['NAMF_C'] = masterPD['NAMF_C'].replace(['DF,MEXICO', 'MEXICO,DF', 'CIUDAD DE MEXICO','DELEGACION BENITO JUAREZ,DF','HIDALGO CIUDAD DE MEXICO', 'DISTRITO NACIONAL', 'OS DISTRITO FEDERAL MEXICO', 'MEXICO, D.F.', 'MEXICO DF', 'DISTRITO FEDERAL', 'MEX. MEXICO', 'CARRANZA,MEXICO DF', 'DELEGACION GUSTAVO A.MADERO DF', 'MEXICO, D.F. 03310', 'MEXICO, D.F. 11510', 'CIUDAD DE MEXICO MEXICO', 'DISTRITO FEDERAL MEXICO', 'MEXICO, D.F. 01219', 'MEXICO, D.F', 'DELEGACION IZTAPALAPA,DF', 'DEL.CUAUHTEMOC, D.F. 06900', 'DELEGACION CUAUHTEMOC DF MEXIC', '97219', 'DELEG. GUSTAVO A. MADERO', 'DELEGACION COYOACAN,DF', 'CUAUHTEMOC,DF', 'DELEGACION GUSTAVO A MADERO,DF', 'MEXICO D.F.', 'DELEGACION CUAUHTEMOC,DF', 'MEXICO, MEXICO', 'MEXICO MEXICO', 'DEL. GUSTAVO A. MADERO', 'DELEGACION CUAJIMALPA,DF', 'DELEGACION CUAUHTEMOC', 'MEX D.F.', 'DELEGACION ALVARO OBREGON,DF', 'DELEGACION MIGUEL HIDALGO DF', 'DEL.GUSTAVO A.MADERO', 'MEX., D.F.', 'DEL. CUAUHTEMOC', 'MEXICO, D.F. 04710', 'EDO DE MEXICO', 'DELEGACION ALVARO OBREGON DF', 'DELEGACION IZTACALCO,DF', 'ESTADO DE MEXICO', 'MEXICO', 'TEXCOCO,MEXICO', 'DEL. AZCAPOTZALCO', 'DEL. BENITO JUAREZ', 'DEL. MIGUEL HIDALGO', 'DEL.CUAUHTEMOC', 'DELEGACION COYOACAN', 'TULTITLAN,MEXICO', 'DEL.VENUSTIANO CARRANZA', 'DELEGACION MIGUEL HIDALGO,DF', 'IXTAPALUCA,MEXICO', 'CHIMALHUACAN ED. DE MEXICO', 'EDO. DE MEXICO', 'CUAUTITLAN IZCALLI MEXICO', 'TLANEPANTLA DE BAZ, MEXICO', 'DELEGACION AZCAPOTZALCO', 'CUAUTITLAN IZCALLI,EDO. DE MEX', 'DEL.TLAHUAC', 'DEL.BENITO JUAREZ', 'DEL.IZTACALCO', 'DEL.AZCAPOTZALCO', 'HIDALGO, MEXICO', 'EDO DE MEX', 'DELEGACION BENITO JUAREZ', 'DEL. VENUSTIANO CARRANZA', 'DEL ALVARO OBREGON', 'DEL. ALVARO OBREGON', 'COACALCO,ESTADO DE MEXICO', 'NAUCALPAN,ESTADO DE MEXICO', 'DEL. BENITO JAUREZ', 'DEL. IZTACALCO', 'DEL.MIGUEL HIDALGO', 'LERMA ESTADO DE MEXICO', 'DEL. BENTIO JAUREZ', 'DEL AZCAPOTZALCO', 'ECATEPEC,ESTADO DE MEXICO', 'DEL. COYOACAN', 'IZTACALCO', 'MIGUEL HIDALGO', '54050', '13280', '53050', '53100', '53370', '52765', '54134', '05100', '07279', '04890', '03800', '03310', '03300', '03100', '03240', '07300', '07570', '07680', '03020', '06400', '02210', '02090', 'C.P 09540', '03810', '11800', '03740', '04320', '11510', '53119', '55720', '14370', '52170', '54130', '54055', '16090', '55080', '52140', '51900', '09360', '52927', '56400', '54508', '08830', '53120'], 'DF')
masterPD['NAMF_C'] = masterPD['NAMF_C'].replace(['JALISCO', 'JAL.', 'ZAPOPAN,JALISCO', 'TEPATITLAN-JALISCO', 'GUADALAJARA,JAL.', 'TLAJOMULCO DE ZUNIGA,JAL.', 'jalisco', 'SAN JUAN DE OCOTAN ZAPOPAN JAL', 'EL SALTO,JALISCO', 'TLQUEPAQUE JAL', 'ZAPOPAN,JALISCO CP.45132', 'JALISCO MEXICO', 'GUADALAJARA,JALISCO', 'PUERTO VALLARTA', '44160', '44940', '44600', '44150', '47095'], 'JAL')
masterPD['NAMF_C'] = masterPD['NAMF_C'].replace(['SLP', 'S.L.P', 'SAN LUIS POTOSI', '78394', '78090'], 'SLP')
masterPD['NAMF_C'] = masterPD['NAMF_C'].replace(['NAYARIT', 'XALISCO-NAYARIT', 'NAY.'], 'NAY')
masterPD['NAMF_C'] = masterPD['NAMF_C'].replace(['GUATEMALA TID 1207317-2'], 'GUATEMALA')
masterPD['NAMF_C'] = masterPD['NAMF_C'].replace(['COAHUILA', 'COA', 'COAH.', 'RAMOS ARIZPE,COAHUILA', 'SALTILLO,COAHUILA', 'C.P 25724', '26250'], 'COA')
masterPD['NAMF_C'] = masterPD['NAMF_C'].replace(['QUINTANA ROO', 'QRO', 'PLAYA DEL CARMEN-QUINTANA ROO', 'Q.ROO', 'PLAYA DEL CARMEN, Q.ROO', 'PLAYA DEL CARMEN QUINTANA ROO', 'QUINTANA ROO MEXICO', 'CANCUN QUINTANA ROO', '77509', '77560'], 'ROO')
masterPD['NAMF_C'] = masterPD['NAMF_C'].replace(['SINALOA', 'CULIACAN,SINALOA', '81200'], 'SIN')
masterPD['NAMF_C'] = masterPD['NAMF_C'].replace(['DURANGO,DURANGO', 'DURANGO', 'GOMEZ PALACIO,DURANGO', 'DGO.', '34000'], 'DGO')
masterPD['NAMF_C'] = masterPD['NAMF_C'].replace(['COSTA RICA', 'SAN JOSE'], 'COSTA RICA')
masterPD['NAMF_C'] = masterPD['NAMF_C'].replace(['BC', 'BAJA CALIFORNIA', 'BAJA CALIFORNIA SUR'], 'BCS')
masterPD['NAMF_C'] = masterPD['NAMF_C'].replace(['SONORA', 'HERMOSILLO,SONORA'], 'SON')
masterPD['NAMF_C'] = masterPD['NAMF_C'].replace(['BAJA CALIFORNIA NORTE', 'C.P 21100', '21100', '21330'], 'BCN')
masterPD['NAMF_C'] = masterPD['NAMF_C'].replace(['NUEVO LAREDO,TAMAULIPAS', 'NUEVO LAREDO,TAMPS.', 'H.MATAMOROS,TAMAULIPAS'], 'TAM')
masterPD['NAMF_C'] = masterPD['NAMF_C'].replace(['CHIHUAHUA,CHIHUAHUA', 'CHIHUAHUA', 'JUAREZ,CHIH.', 'CHIH.', '31050', '31110', '32690'], 'CHH')
masterPD['NAMF_C'] = masterPD['NAMF_C'].replace(['SAO PAULO, BRASIL'], 'BRASIL')
masterPD['NAMF_C'] = masterPD['NAMF_C'].replace(['QUERETARO', 'QUERETARO, QUERETARO', 'QRO.', '76110', '76090', '76900'], 'QUE')
masterPD['NAMF_C'] = masterPD['NAMF_C'].replace(['STA MARIA HUATULCO, OAX 70989', 'OAXACA', 'SANTA CRUZ HUATULCO.OAX.', '71980'], 'OAX')
masterPD['NAMF_C'] = masterPD['NAMF_C'].replace(['PUEBLA', 'HUEJOTZINGO,PUEBLA', '72410', '72140'], 'PUE')
masterPD['NAMF_C'] = masterPD['NAMF_C'].replace(['THE UNITED ARAB EMIRATES'], 'UAE')
masterPD['NAMF_C'] = masterPD['NAMF_C'].replace(['REPUBLICA DE PANAMA'], 'PANAMA')
masterPD['NAMF_C'] = masterPD['NAMF_C'].replace(['FL,USA'], 'FLORIDA')
masterPD['NAMF_C'] = masterPD['NAMF_C'].replace(['CAMPECHE, CAMPECHE.', 'CAMPECHE'], 'CAM')
masterPD['NAMF_C'] = masterPD['NAMF_C'].replace(['AGS.'], 'AGU')
masterPD['NAMF_C'] = masterPD['NAMF_C'].replace(['GUANAJUATO', 'SILAO,GUANAJUATO', 'GTO.', '36631', '37530'], 'GUA')
masterPD['NAMF_C'] = masterPD['NAMF_C'].replace(['VERACRUZ', 'VERACRUZ MEXICO', 'VER.', 'ACAYUACAN,VERACRUZ', '94300'], 'VER')
masterPD['NAMF_C'] = masterPD['NAMF_C'].replace(['PROGRESO,YUCATAN', 'MERIDA,YUCATAN', 'MERIDA , YUCATAN', 'YUCATAN', 'MERIDA YUCATAN'], 'YUC')
masterPD['NAMF_C'] = masterPD['NAMF_C'].replace(['CARACAS, VENEZUELA'], 'VENEZUELA')
masterPD['NAMF_C'] = masterPD['NAMF_C'].replace(['ZACATECAS'], 'ZAC')
masterPD['NAMF_C'] = masterPD['NAMF_C'].replace(['TLAXCALA'], 'TLA')
masterPD['NAMF_C'] = masterPD['NAMF_C'].replace(['GUERRERO'], 'GRO')
masterPD['NAMF_C'] = masterPD['NAMF_C'].replace(['CHIAPAS'], 'CHP')
masterPD['NAMF_C'] = masterPD['NAMF_C'].replace(['MANAGUA'], 'NICARAGUA')
masterPD['NAMF_C'] = masterPD['NAMF_C'].replace(['COLIMA', 'COLIMA,COLIMA', '28047'], 'COL')
masterPD['NAMF_C'] = masterPD['NAMF_C'].replace(['TABASCO', 'MACUSPANA,TABASCO', 'TAB', '86097'], 'TAB')
masterPD['NAMF_C'] = masterPD['NAMF_C'].replace(['JOJUTLA,MORELOS', 'MORELOS', 'CUERNAVACA,MORELOS', '62590'], 'MOR')
masterPD['NAMF_C'] = masterPD['NAMF_C'].replace(['HIDALGO', 'HIDALGO MEXICO', '42160'], 'HID')
masterPD['NAMF_C'] = masterPD['NAMF_C'].replace(['TAMAULIPAS', '88710', '87149', '87180'], 'TAM')
masterPD['NAMF_C'] = masterPD['NAMF_C'].replace(['MICHOACAN', 'C.P 52279'], 'MIC')
masterPD['NAMF_C'] = masterPD['NAMF_C'].replace(['CUNDINAMARCA, COLOMBIA'], 'COLOMBIA')
masterPD['NAMF_C'] = masterPD['NAMF_C'].replace(['22430'], 'TIJ')
# removing postal codes that had an input of 1
masterPD['PSTC_C'] = masterPD['PSTC_C'].replace([1, '1'], np.nan)
# cleaning the column CREG_OV
masterPD['CREG_OV'] = masterPD['CREG_OV'].replace(['DFE', 'D.F'], 'DF')
masterPD['CREG_OV'] = masterPD['CREG_OV'].replace(['QNR'], 'ROO')
masterPD['CREG_OV'] = masterPD['CREG_OV'].replace(['Nay'], 'NAY')
masterPD['CREG_OV'] = masterPD['CREG_OV'].replace(['Jal'], 'JAL')
masterPD['CREG_OV'] = masterPD['CREG_OV'].replace(['Gto'], 'GUA')
masterPD['CREG_OV'] = masterPD['CREG_OV'].replace(['TX.'], 'TX')
masterPD['CREG_OV'] = masterPD['CREG_OV'].replace(['BAJ'], 'BCN')
masterPD['CREG_OV'] = masterPD['CREG_OV'].replace(['HGO', 'Hid'], 'HID')
masterPD['CREG_OV'] = masterPD['CREG_OV'].replace(['Pue'], 'PUE')
masterPD['CREG_OV'] = masterPD['CREG_OV'].replace(['Gua'], 'GUA')
masterPD['CREG_OV'] = masterPD['CREG_OV'].replace(['Coa'], 'COA')
masterPD['CREG_OV'] = masterPD['CREG_OV'].replace(['Mic'], 'MIC')
masterPD['CREG_OV'] = masterPD['CREG_OV'].replace(['PR'], 'PUERTO RICO')
masterPD['CREG_OV'] = masterPD['CREG_OV'].replace(['EUR'], 'EUROPA')
masterPD['CREG_OV'] = masterPD['CREG_OV'].replace(['Son'], 'SON')
masterPD['CREG_OV'] = masterPD['CREG_OV'].replace(['Yuc'], 'YUC')
masterPD['CREG_OV'] = masterPD['CREG_OV'].replace(['Ags'], 'AGS')
masterPD['CREG_OV'] = masterPD['CREG_OV'].replace(['Zac'], 'ZAC')
masterPD['CREG_OV'] = masterPD['CREG_OV'].replace(['QRO'], 'ROO') # este puede ser queretaro o quintana roo
masterPD['CREG_OV'] = masterPD['CREG_OV'].replace(['N.L'], 'NLE')
masterPD['CREG_OV'] = masterPD['CREG_OV'].replace(['col'], 'COL')
masterPD['CREG_OV'] = masterPD['CREG_OV'].replace(['Gro'], 'GRO')
masterPD['CREG_OV'] = masterPD['CREG_OV'].replace(['Mor'], 'MOR')
masterPD['CREG_OV'] = masterPD['CREG_OV'].replace(['Que'], 'QUE')

In [0]:
# cambiar el tipo de dataframe para poder guardar
masterLimpia = spark.createDataFrame(masterPD)
Save_Table_Sustituir(masterLimpia,"DataSet","masterV4L","/mnt/delta/drvdev/")

Tabla en BD: DataSet.masterV4L
Out[18]: 'DataSet.masterV4L'

In [0]:
# EDA de la tabla maestra limpia V4
#from pandas_profiling import ProfileReport
#profile = ProfileReport(masterPD, title="Pandas Profiling Report", explorative=True, correlations={"kendall": {"calculate": False}, "phi_k": {"calculate": False},"cramers": {"calculate": False}}, interactions={'continuous': False}, missing_diagrams={"bar": False})
#profile.to_file('profile.html')

In [0]:
# mostrar el EDA
#displayHTML(profile.html)